# This file runs on ***Google Colab***
## Before running this file, the Data Source file needs to be placed at the same level as this file

### Data file location: https://github.com/JagpreetBath/European_Hotel_Analysis/tree/main/DataFiles/TransformedData/tables/hotel_reviews_for_NLP2_int_review_score.csv

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (142 kB/s)
Reading package lists... Done


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark_NLP_Attempt1").getOrCreate()

In [3]:
# Read in data 
from pyspark import SparkFiles
file_path = "hotel_reviews_for_NLP2_int_review_score.csv"
spark.sparkContext.addFile(file_path)
df = spark.read.csv(SparkFiles.get("hotel_reviews_for_NLP2_int_review_score.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+--------------+
|              Review|Reviewer_Score|
+--------------------+--------------+
| I am so angry th...|             2|
| No real complain...|             7|
| Rooms are nice b...|             7|
| My room was dirt...|             3|
| You When I booke...|             6|
| Backyard of the ...|             6|
| Cleaner did not ...|             4|
| Apart from the p...|            10|
| Even though the ...|             6|
| The aircondition...|             7|
| Nothing all grea...|            10|
| 6 30 AM started ...|             5|
| The floor in my ...|             4|
| This hotel is be...|             9|
| The staff in the...|             8|
| This hotel is aw...|            10|
| Very steep steps...|             6|
| We did not like ...|             7|
| Public areas are...|             7|
| We had issues wi...|             7|
+--------------------+--------------+
only showing top 20 rows



In [4]:
# Import functions
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [5]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature
data_df = df.withColumn('length', length(df['Review']))
data_df.show()

+--------------------+--------------+------+
|              Review|Reviewer_Score|length|
+--------------------+--------------+------+
| I am so angry th...|             2|  1913|
| No real complain...|             7|   611|
| Rooms are nice b...|             7|   301|
| My room was dirt...|             3|  1221|
| You When I booke...|             6|   774|
| Backyard of the ...|             6|   186|
| Cleaner did not ...|             4|   235|
| Apart from the p...|            10|   157|
| Even though the ...|             6|   162|
| The aircondition...|             7|   312|
| Nothing all grea...|            10|   568|
| 6 30 AM started ...|             5|   430|
| The floor in my ...|             4|   152|
| This hotel is be...|             9|   329|
| The staff in the...|             8|   229|
| This hotel is aw...|            10|   413|
| Very steep steps...|             6|   270|
| We did not like ...|             7|   623|
| Public areas are...|             7|   166|
| We had i

In [6]:
# Create all the features to the data set
convert_review_output_to_numbers = StringIndexer(inputCol='Reviewer_Score',outputCol='label')
tokenizer = Tokenizer(inputCol="Review", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [8]:
# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[convert_review_output_to_numbers, tokenizer, stopremove, hashingTF, idf, clean_up])

In [9]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

# Show "Combined_Review" and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  8.0|(262145,[2437,302...|
|  2.0|(262145,[4714,514...|
|  2.0|(262145,[22346,23...|
|  7.0|(262145,[1797,230...|
|  4.0|(262145,[14870,20...|
|  4.0|(262145,[9781,304...|
|  6.0|(262145,[21641,34...|
|  1.0|(262145,[25789,43...|
|  4.0|(262145,[22815,31...|
|  2.0|(262145,[2437,216...|
|  1.0|(262145,[9129,181...|
|  5.0|(262145,[1696,383...|
|  6.0|(262145,[1729,216...|
|  0.0|(262145,[15370,23...|
|  3.0|(262145,[6957,304...|
|  1.0|(262145,[5765,218...|
|  4.0|(262145,[3280,110...|
|  2.0|(262145,[329,9129...|
|  2.0|(262145,[11941,17...|
|  2.0|(262145,[17435,21...|
+-----+--------------------+
only showing top 20 rows



In [10]:
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3], 21)

In [11]:
from pyspark.ml.classification import NaiveBayes
# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [12]:
# Transform the data with the testing data
test_results = predictor.transform(testing)
test_results.show(10)

+--------------------+--------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|              Review|Reviewer_Score|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| A bit noisy You ...|             8|   429|  3.0|[, a, bit, noisy,...|[, bit, noisy, he...|(262144,[9781,216...|(262144,[9781,216...|(262145,[9781,216...|[-2613.6665944248...|[1.0,6.9364358196...|       0.0|
| A very well orga...|             8|   123|  3.0|[, a, very, well,...|[, well, organise...|(262144,[6346,732...|(262144,[6346,732...|(262145,[6346,732...|[-521.521

In [13]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" %acc)

Accuracy of model at predicting reviews was: 0.118251
